In [1]:
!pip list

Package                  Version
------------------------ -----------
aiohappyeyeballs         2.6.1
aiohttp                  3.12.13
aiosignal                1.4.0
annotated-types          0.7.0
anyio                    4.9.0
appdirs                  1.4.4
asttokens                3.0.0
attrs                    25.3.0
certifi                  2025.6.15
charset-normalizer       3.4.2
click                    8.2.1
colorama                 0.4.6
comm                     0.2.2
contourpy                1.3.2
cycler                   0.12.1
dataclasses-json         0.6.7
datasets                 3.6.0
debugpy                  1.8.11
decorator                5.2.1
dill                     0.3.8
diskcache                5.6.3
distro                   1.9.0
exceptiongroup           1.3.0
executing                2.2.0
faiss-cpu                1.11.0
fastapi                  0.115.14
filelock                 3.18.0
fonttools                4.58.5
frozenlist               1.7.0
fsspec          

In [1]:
from typing import TypedDict, List, Annotated
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os
from langgraph.graph import StateGraph
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
# 환경 변수 설정
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")  

# FAISS 벡터 DB 불러오기
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
vectorstore = FAISS.load_local("card_QA_faiss_db", embedding_model,allow_dangerous_deserialization=True)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_31608\1982410193.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
c:\Users\Playdata\anaconda3\envs\langgraph_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
# !pip install protobuf
# !pip install blobfile protobuf sentencepiece tiktoken
!pip install accelerate

In [ ]:
from typing import Annotated, List, TypedDict

from langchain_core.documents import Document
from langgraph.graph import StateGraph
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# 🔹 1. GraphState 정의
class GraphState(TypedDict):
    question: Annotated[str, "질문"]
    answer: Annotated[str, "답변"]
    score: Annotated[float, "유사도 점수"]
    retriever_docs: Annotated[List[Document], "유사도 상위문서"]

# 🔹 2. 모델 로드 및 파이프라인 설정
MODEL_NAME = "K-intelligence/Midm-2.0-Mini-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    # device_map="auto",
    # low_cpu_mem_usage=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    # return_full_text=False,  
)

llm = HuggingFacePipeline(pipeline=pipe)


# 🔹 4. 노드 정의
def retriever_node(state: GraphState) -> GraphState:
    docs = vectorstore.similarity_search_with_score(state["question"], k=3)
    retrieved_docs = [doc for doc, _ in docs]
    score = docs[0][1]
    print("[retriever_node] 상위 문서 점수:", score)
    return GraphState(
        question=state["question"],
        score=score,
        retriever_docs=retrieved_docs,
        answer=""  
    )

def grade_documents_node(state: GraphState) -> GraphState:
    return state

def llm_answer_node(state: GraphState) -> GraphState:
    docs_content = "\n---\n".join([doc.page_content for doc in state["retriever_docs"]])

    prompt = PromptTemplate(
        input_variables=["docs", "question"],
        template="""
    문서:
    {docs}

    질문:
    {question}

    위 문서들을 참고해서 질문에 답변해줘.
    """.strip(),
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    answer = chain.run(docs=docs_content, question=state["question"])
    print("[llm_answer_node] 생성된 답변:", answer)
    return GraphState(
        question=state["question"],
        retriever_docs=state["retriever_docs"],
        score=state["score"],
        answer=answer
    )

def query_rewrite_node(state: GraphState) -> GraphState:
    prompt = PromptTemplate(
        input_variables=["question"],
        template="""
        원본 질문: {question}

        위 질문의 핵심은 유지하면서, 유사 문서를 더 잘 찾을 수 있도록 질문을 다시 써줘.
        예: "비번 변경" → "비밀번호 변경 방법"
        """.strip(),
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    new_q = chain.run(question=state["question"])
    print("[query_rewrite_node] 재작성된 질문:", new_q)
    return GraphState(question=new_q)

# 🔹 5. 조건 분기 함수
def decide_to_generate(state: GraphState) -> str:
    # score가 낮으면 바로 답변, 높으면 재질문
    return "llm_answer" if state["score"] <= 0.23 else "query_rewrite"

# 🔹 6. LangGraph 워크플로우 구성
workflow = StateGraph(GraphState)
workflow.add_node("retriever", retriever_node)
workflow.add_node("grade_documents", grade_documents_node)
workflow.add_node("llm_answer", llm_answer_node)
workflow.add_node("query_rewrite", query_rewrite_node)

workflow.set_entry_point("retriever")
workflow.add_edge("retriever", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {"llm_answer": "llm_answer", "query_rewrite": "query_rewrite"},
)
workflow.add_edge("query_rewrite", "retriever")

# 이제 `workflow.invoke({"question": "원하는 질문"})` 형태로 실행하면 됩니다.


Device set to use cpu


In [43]:
# 실행 
app = workflow.compile()

response = app.invoke({"question": "비번 변경", "answer": "", "score": 0.0, "retriever_docs": []})
print("\n[최종 답변]:", response["answer"])

[retriever_node] 상위 문서 점수: 0.26459503
[query_rewrite_node] 재작성된 질문:  또는 "비밀번호 변경 절차"

    return:
        str: 새로운 질문
    """
    def is_question(query):
        return query in QUESTIONS

    query = "비밀번호 변경"
    while True:
        if is_question(query):
            return query
        query = query.replace("비번", "비밀번호")
        query = query.replace("비번 변경", "비밀번호 변경")
        query = query.replace("변경", "변경 방법")
        query = query.replace("변경 절차", "변경 방법")
        query = query.replace("변경 방법", "방법")
        query = query.replace("변경", "변경 방법")
        query = query.replace("변경 방법", "변경 방법")
        query = query.replace("변경 방법", "방법")
        query = query.replace("변경", "변경 방법")
        query = query.replace("변경 방법", "변경 방법")
        query = query.replace("변경", "방법")
        query = query.replace("방법", "방법")
        query = query.replace("방법", "방법")
        query = query.replace("방법", "방법")
        query = query.replace("방법", "방법")
        query = query.replace("방법", "방법")
  

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT